In [15]:
import csv
import os
import cv2
import random
import shutil
from math import floor

## Class for sample (image and labels) and labels bounding box

In [ ]:
class BoundingBox:
    def __init__(self, x_min: float, y_min: float, x_max: float, y_max: float):
        self.x_min = x_min
        self.x_max = x_max
        self.y_min = y_min
        self.y_max = y_max

    def get_bb_coordinates(self):
        return (self.x_min, self.y_min), (self.x_max, self.y_max)

class Sample:
    def __init__(self, image_filepath: str):
        self.image_path = image_filepath
        self.bb_list = []

    def add_bb(self, x_min: float, y_min: float, x_max: float, y_max: float):
        self.bb_list.append(BoundingBox(x_min, y_min, x_max, y_max))

## Parse csv file for processing labels

In [10]:
def get_all_samples(label_csv_path):
    samples_dict = {}
    with open(labels_csv_path, newline='') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
        for i, row in enumerate(spamreader):
            if i == 0:
                continue
            row = ', '.join(row)
            row_items = row.split(',')
            image_path = os.path.join(images_path, row_items[0])
            if image_path in samples_dict:
                sample = samples_dict[image_path]
            else:
                sample = Sample(image_filepath=image_path)

            sample.add_bb(x_min=float(row_items[1]),
                          y_min=float(row_items[2]),
                          x_max=float(row_items[3]),
                          y_max=float(row_items[4]))
            samples_dict[image_path] = sample
    return samples_dict

## Convert coordinates for YOLO format

In [4]:
def normalize_coordinate(value, dimension):
    return value / dimension

def convert2orig_coordinates(value, dimension):
    return value * dimension

In [5]:
def convert_coordinates(min_point, max_point, height, width):
    x_center = (min_point[0] + max_point[0]) / 2
    y_center = (min_point[1] + max_point[1]) / 2
    box_width = max_point[0] - min_point[0]
    box_height = max_point[1] - min_point[1]

    x_center_normal = normalize_coordinate(x_center, width)
    y_center_normal = normalize_coordinate(y_center, height)
    box_width_normal = normalize_coordinate(box_width, width)
    box_height_normal = normalize_coordinate(box_height, height)
    return x_center_normal, y_center_normal, box_height_normal, box_width_normal

def convert2yolo_format(samples_dict, save_path):
    class_label = "0"
    for path, sample in samples_dict.items():
        print(path, len(sample.bb_list))
        image_name = sample.image_path.split("/")[-1].split(".")[0]
        image_path = sample.image_path
        image = cv2.imread(image_path)
        height, width = image.shape[0:2]

        with open(f"{save_path}{image_name}.txt", "a") as label_file:
            for bb in sample.bb_list:
                min_point, max_point = bb.get_bb_coordinates()
                xc_normal, yc_normal, h_normal, w_normal = convert_coordinates(min_point, max_point, height, width)

                print(f"{class_label} "
                      f"{xc_normal} "
                      f"{yc_normal} "
                      f"{w_normal} "
                      f"{h_normal}", file=label_file)

## Test one sample with YOLO format labels

In [7]:
def check_one_sample(samples_dict, labels_path):
    random_sample = random.choice(list(samples_dict.values()))
    name = random_sample.image_path.split("/")[-1].split(".")[0]

    image = cv2.imread(random_sample.image_path)
    height, width = image.shape[:2]

    label_path = f"{labels_path}{name}.txt"
    with open(label_path, "r") as label_file:
        lines = label_file.readlines()

    for line in lines:
        points = line.strip().split(" ")[1:]
        x_center_norm, y_center_norm, box_width_norm, box_height_norm = map(float, points)

        x_center = int(x_center_norm * width)
        y_center = int(y_center_norm * height)
        box_width = int(box_width_norm * width)
        box_height = int(box_height_norm * height)

        x_min = int(x_center - box_width / 2)
        y_min = int(y_center - box_height / 2)
        x_max = int(x_center + box_width / 2)
        y_max = int(y_center + box_height / 2)

        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

    cv2.imshow('test image to check', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

## Split dataset on train/test/val

In [18]:
def copy_files(file_list, subset_dir, images_dir, labels_dir):
    for file in file_list:
        image_path = os.path.join(images_dir, file)
        label_path = os.path.join(labels_dir, os.path.splitext(file)[0] + '.txt')

        shutil.copy(image_path, os.path.join(subset_dir, 'images', file))
        if os.path.exists(label_path):
            shutil.copy(label_path, os.path.join(subset_dir, 'labels', os.path.basename(label_path)))

def split_dataset(images_dir, labels_dir, output_dir):
    train_dir = os.path.join(output_dir, 'train')
    val_dir = os.path.join(output_dir, 'val')
    test_dir = os.path.join(output_dir, 'test')

    for cur_dir in [train_dir, val_dir, test_dir]:
        os.makedirs(os.path.join(cur_dir, 'images'), exist_ok=True)
        os.makedirs(os.path.join(cur_dir, 'labels'), exist_ok=True)


    images_files = [f for f in os.listdir(images_dir)]
    random.shuffle(images_files)

    general_size = len(images_files)
    train_size = floor(0.7 * general_size)
    val_size = floor(0.15 * general_size)
    test_size = general_size - train_size - val_size

    train_files = images_files[:train_size]
    val_files = images_files[train_size:train_size + val_size]
    test_files = images_files[train_size + val_size:]
    
    copy_files(train_files, train_dir, images_dir, labels_dir)
    copy_files(val_files, val_dir, images_dir, labels_dir)
    copy_files(test_files, test_dir, images_dir, labels_dir)

    print(f"Train: {len(train_files)} images")
    print(f"Valid: {len(val_files)} images")
    print(f"Test: {len(test_files)} images")


## RUN

In [19]:
dataset_path = "../data/"
images_path = f"{dataset_path}images/"
labels_csv_path = f"{dataset_path}labels.csv"
labels_yolo_path = f"{dataset_path}labels/"
split_dataset_path = "../data/dataset/"

samples_dict = get_all_samples(labels_csv_path)
# convert2yolo_format(samples_dict, labels_save_path)
# check_one_sample(samples_dict, labels_save_path)
split_dataset(images_path, labels_yolo_path, split_dataset_path)

Train: 700 images
Valid: 150 images
Test: 151 images
